In [ ]:
# Running %env without any arguments
# lists all environment variables

# The line below sets the environment
# variable CUDA_VISIBLE_DEVICES
%env CUDA_VISIBLE_DEVICES = 0

import numpy as np
import pandas as pd
import io
import time
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from scipy.misc import imread, imsave, imshow
import tensorflow as tf
from tensorflow.python.platform import tf_logging
from tensorflow.contrib import layers
from tensorflow.contrib.training import add_gradients_summaries
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import nn_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.training import optimizer as tf_optimizer
from tensorflow.python.ops import variables as tf_variables
import os.path
import tensorflow.contrib.slim as slim
import inception_preprocessing
import vgg_preprocessing
import logging
import resnet2

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
# last count: 9817, roughly examples num: 10052608
# after resample count: 18456575

# '/media/rs/FC6CDC6F6CDC25E4/resample_dataset2/'
# total sampled examples num: 18456575

In [3]:
# after epoches 3, 59% at val
# change mode test
# lower weight of aux: 0.1 -> 0.05, 0.25 -> 0.15
# hard num: 3071437
# after hard: 19.732, on val: 36%, step: 271380
# after hard: change to momentum, main label_smoothing: 0.1->0.05, weight: 0.15->0.1, 0.05->0.03
# after finetuning 2(375933): change momentum: 0.8->0.4, weight: ->0, ->0, ACCUMULATE_STEP:->1, change dataset: ->balance

In [4]:
DATASET_PATH = '/media/rs/0E06CD1706CD0127/Kapok/kaggle/'
PRETRAINED_MODEL_PATH = DATASET_PATH + 'Resnet/logs101-new/model_finetune/resnet101_v2_model.ckpt-375933'
#PRETRAINED_MODEL_PATH = DATASET_PATH + 'Resnet/resnet2/resnet_v2_101.ckpt'
LOG_PATH = DATASET_PATH + 'Resnet/logs101-new/'
#TRAIN_PATH = DATASET_PATH + 'Split1/Train/'
TRAIN_PATH = '/media/rs/FC6CDC6F6CDC25E4/resample_dataset2/'
#TRAIN_PATH = '/media/rs/FC6CDC6F6CDC25E4/ResnetHardTrain/'
VAL_PATH = DATASET_PATH + 'Split1/Validation/'
TEST_PATH = DATASET_PATH + 'Test/'
CATEGORY_NAME_PATH = DATASET_PATH + 'category_names.csv'
BATCH_SIZE = 128#256

# total_batch_size is BATCH_SIZE * ACCUMULATE_STEP
ACCUMULATE_STEP = 1

IMAGE_WIDTH = 180
IMAGE_HEIGHT = 180
NUM_CLASS = 5270
LEVEL0_CLASS = 49
LEVEL1_CLASS = 483
# validation examples num: 2319624
# train examples num: 10051704
# total step: 157057
# start 141310, change balance
TOTAL_EXAMPLES = 18456575

NUM_EPOCHES = 12
EPOCHES_OVER = 10

INPUT_THREADS = 12

#Learning rate information and configuration (Up to you to experiment)
# initial_learning_rate = 0.000003#0.00001
# learning_rate_decay_factor = 0.94
initial_learning_rate = 0.0002#0.0004
stop_learning_rate = 0.000002
learning_rate_decay_factor = 0.95
num_epochs_before_decay = 1
moving_average_decay = 0.96# use large to be more stable?
momentum = 0.4
#Know the number steps to take before decaying the learning rate and batches per epoch
num_steps_per_epoch = TOTAL_EXAMPLES / (BATCH_SIZE * ACCUMULATE_STEP) + 1
decay_steps = int(num_epochs_before_decay * num_steps_per_epoch / 6)

In [5]:
# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler(DATASET_PATH + 'tensorflow_resnet_train.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)

In [6]:
def preprocess_for_inception(input_image, is_training = True):
    return inception_preprocessing.preprocess_image(input_image, 160, 160, is_training)

In [7]:
class LabelMapping(object):
    def __init__(self, catogory_file_path):
        super(LabelMapping, self).__init__()
        self._category_level_csv = catogory_file_path
        self._category_map, self._category_level0_map, self._category_level1_map, self._len_level0, self._len_level1 = self.cvt_csv2tfrecord()
        self._mapping_strings = tf.constant( [ str(key) for key in self._category_map.keys() ] )
        #print(list(self._category_map.keys())[0])
        self._mapping_table = tf.contrib.lookup.index_table_from_tensor(mapping=self._mapping_strings, default_value=0) 
        
        self._level0_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(self._category_level0_map.keys()), list(self._category_level0_map.values()), tf.int64, tf.int64), 0)
        self._level1_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(self._category_level1_map.keys()), list(self._category_level1_map.values()), tf.int64, tf.int64), 0)

    @property
    def category_map(self):
        return self._category_map
    @property
    def level0_table(self):
        return self._level0_table
    @property
    def level1_table(self):
        return self._level1_table
    @property
    def len_level0(self):
        return self._len_level0
    @property
    def len_level1(self):
        return self._len_level1
    @property
    def mapping_table(self):
        return self._mapping_table
    
    def cvt_csv2tfrecord(self):
        level0_map, level1_map = self.create_level_map()
        count = 0
        category_map = dict()
        category_level0_map = dict()
        category_level1_map = dict()
        csv = pd.read_csv(self._category_level_csv).values
        for row in csv:  
            category_id, level0, level1 = row[0], row[1], row[2]
            category_map[category_id] = count
            category_level0_map[int(category_id)] = level0_map[level0]
            category_level1_map[int(category_id)] = level1_map[level1]
            count += 1

        return category_map, category_level0_map, category_level1_map, len(level0_map), len(level1_map)

    def create_level_map(self):
        csv = pd.read_csv(self._category_level_csv).values
        level_list = [list(), list()]
        for row in csv: 
            for level in range(1,3):
                if row[level] not in level_list[level-1]:
                    level_list[level-1].append(row[level])
        return dict(zip(level_list[0], range(len(level_list[0])))), dict(zip(level_list[1], range(len(level_list[1]))))

In [8]:
class CdiscountDataset(object):
    def __init__(self, data_path, file_begin_match, label_mapping, num_examples, num_classes, buffer_size, batch_size, num_epochs, is_training):
        super(CdiscountDataset, self).__init__()
        #self._data_file_list = [ os.path.join(data_path, x) for x in os.listdir(data_path) if lambda x: os.path.isfile(x) and x.startswith(file_begin_match) ]
        self._data_file_list = data_path + file_begin_match + '*'
        self._num_examples = num_examples
        self._num_classes = num_classes
        self._batch_size = batch_size
        self._buffer_size = buffer_size
        self._num_epochs = num_epochs
        self._is_training = is_training
        self._category_map = label_mapping.category_map
        self._level0_table = label_mapping.level0_table
        self._level1_table = label_mapping.level1_table
        self._len_level0 = label_mapping.len_level0
        self._len_level1 = label_mapping.len_level1
        self._mapping_table = label_mapping.mapping_table
    
    
    def create_dataset(self):
        opts = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.ZLIB)
        reader = lambda : tf.TFRecordReader(options=opts)
        keys_to_features = {
            'img_raw': tf.FixedLenFeature([], tf.string, default_value=''),
            'product_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64)),
            # notice that we don't have this feature in our TFRecord, so always default provided
            'format': tf.FixedLenFeature([], tf.string, default_value='jpg'),
            'category_id': tf.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
        }

        items_to_handlers = {
            # automated decode image from features in FixedLenFeature
            'image': slim.tfexample_decoder.Image(image_key='img_raw', format_key='format'),
            'label': slim.tfexample_decoder.Tensor('category_id'),
        }

        decoder = slim.tfexample_decoder.TFExampleDecoder(keys_to_features, items_to_handlers)
        
        self._dataset = slim.dataset.Dataset(
            data_sources = self._data_file_list,
            decoder = decoder,
            reader = reader,
            # num_readers = 8,
            num_samples = self._num_examples,
            #num_classes = self._num_classes,
            items_to_descriptions = None)
        
        # notice that DatasetDataProvider can automate shuffle the examples by ParallelReader using its RandomShuffleQueue
        self._data_provider = slim.dataset_data_provider.DatasetDataProvider(
            self._dataset,
            num_readers = INPUT_THREADS,
            shuffle = True, # default is True
            num_epochs = self._num_epochs,
            common_queue_capacity = self._buffer_size + 4 * self._batch_size,
            common_queue_min = self._buffer_size,
            scope = self._is_training and 'train_files' or 'validation_files')
        
        org_image, org_label = self._data_provider.get(['image', 'label'])

        image = preprocess_for_inception(org_image, self._is_training) # final image to train
 
        # no need for shuffle, DatasetDataProvider do this for us
        batch_images, batch_labels, batch_labels_level0, batch_labels_level1 = \
                tf.train.batch([image, tf.one_hot(self._mapping_table.lookup(tf.as_string(org_label)), self._num_classes, axis=-1),\
                tf.one_hot(self._level0_table.lookup(org_label), self._len_level0, axis=-1),\
                tf.one_hot(self._level1_table.lookup(org_label), self._len_level1, axis=-1)],\
                self._batch_size,\
                num_threads = INPUT_THREADS,\
                capacity = self._buffer_size + 4 * self._batch_size,\
                allow_smaller_final_batch = self._is_training, name = self._is_training and 'train_batch' or 'validation_batch')
        
        return batch_images, batch_labels, batch_labels_level0, batch_labels_level1

In [9]:
def my_create_train_op(total_loss, optimizer, summarize_gradients = False, accumulate_factor=None):
    global_step = tf.train.get_or_create_global_step()

    update_ops = set(ops.get_collection(ops.GraphKeys.UPDATE_OPS))

    # Make sure update_ops are computed before total_loss.
    if update_ops:
        with ops.control_dependencies(update_ops):
            barrier = control_flow_ops.no_op(name='update_barrier')
    total_loss = control_flow_ops.with_dependencies([barrier], total_loss)

    variables_to_train = tf_variables.trainable_variables()

    # initialized with 0s
    accum_vars = [tf.Variable(tf.zeros_like(tv.initialized_value()), trainable=False) for tv in variables_to_train]
    zero_ops = [tv.assign(tf.zeros_like(tv)) for tv in accum_vars]

    # Calls the compute_gradients function of the optimizer to obtain... the list of gradients
    grads = optimizer.compute_gradients(
      total_loss,
      variables_to_train,
      gate_gradients=tf_optimizer.Optimizer.GATE_OP,
      aggregation_method=None,
      colocate_gradients_with_ops=False)

    ## Adds to each element from the list you initialized earlier with zeros its gradient (works because accum_vars and grads are in the same order)
    if accumulate_factor is not None:   
        total_loss = array_ops.check_numerics(total_loss, 'LossTensor is inf or nan')
        with tf.control_dependencies([total_loss]):
            accum_ops = [accum_vars[i].assign_add(gv[0]) for i, gv in enumerate(grads) if gv[0] is not None]

        ## Define the training step (part with variable value update)
        accumulate_grads = [(tf.multiply(accum_vars[i], accumulate_factor), gv[1]) for i, gv in enumerate(grads) if gv[0] is not None]
    else:
        accum_ops = tf.no_op(name = 'accum_pass_by')
    
    if accumulate_factor is not None: 
        # Summarize gradients.
        if summarize_gradients:
            with ops.name_scope('summarize_grads'):
                add_gradients_summaries(accumulate_grads)
        grad_updates = optimizer.apply_gradients(accumulate_grads, global_step=global_step)
    else:
        # Summarize gradients.
        if summarize_gradients:
            with ops.name_scope('summarize_grads'):
                add_gradients_summaries(grads)
        grad_updates = optimizer.apply_gradients(grads, global_step=global_step)

    with ops.name_scope('train_op'):
        # Ensure the train_tensor computes grad_updates.
        train_op = control_flow_ops.with_dependencies([grad_updates], total_loss)

    # Add the operation used for training to the 'train_op' collection
    train_ops = ops.get_collection_ref(ops.GraphKeys.TRAIN_OP)
    if train_op not in train_ops:
        train_ops.append(train_op)

    return train_op, accum_ops, zero_ops

In [10]:
def_graph = tf.Graph()
with def_graph.as_default() as graph:
    def my_exponential_decay(learning_rate, global_step, decay_steps, decay_rate,
                      staircase=False, name=None):
        if global_step is None:
            raise ValueError("global_step is required for exponential_decay.")
        with ops.name_scope(name, "ExponentialDecay",
                          [learning_rate, global_step,
                           decay_steps, decay_rate]) as name:
            learning_rate = ops.convert_to_tensor(learning_rate, name="learning_rate")
            dtype = learning_rate.dtype
            global_step = math_ops.cast(global_step, dtype)
            decay_steps = math_ops.cast(decay_steps, dtype)
            decay_rate = math_ops.cast(decay_rate, dtype)
            p = global_step / decay_steps
            if staircase:
                p = math_ops.floor(p)
            return tf.maximum(math_ops.multiply(learning_rate, math_ops.pow(decay_rate, p),
                                 name=name), tf.constant(stop_learning_rate, dtype=dtype))
    def train_step(input_examples, one_hot_labels, level0_labels, level1_labels):   
        # inputs has shape [batch, 224, 224, 3]
        with slim.arg_scope(resnet2.resnet_arg_scope()):
            logits, end_points = resnet2.resnet_v2_101(input_examples, NUM_CLASS, is_training=True)
            
            variables_to_restore = slim.get_variables_to_restore(exclude = ['resnet_v2_101/logits'])

            level0_aux_input = def_graph.get_tensor_by_name('resnet_v2_101/block3/unit_23/bottleneck_v2/add:0')
            level1_aux_input = def_graph.get_tensor_by_name('resnet_v2_101/block4/unit_3/bottleneck_v2/add:0')   
           
            level0_aux_input = layers.batch_norm(
                level0_aux_input, activation_fn=nn_ops.relu, scope='postnorm_aux0')
            
            level1_aux_input = layers.batch_norm(
                level1_aux_input, activation_fn=nn_ops.relu, scope='postnorm_aux1')
        
            level0_aux_net = math_ops.reduce_mean(level0_aux_input, [1, 2], name='level0_aux_pool', keep_dims=True)
            level0_aux_net = layers.conv2d(
                  level0_aux_net,
                  LEVEL0_CLASS, [1, 1],
                  activation_fn=None,
                  normalizer_fn=None,
                  scope='level0_aux_conv')
            level0_aux_logits = tf.squeeze(level0_aux_net)
            loss_level0 = tf.losses.softmax_cross_entropy(onehot_labels = level0_labels, logits = level0_aux_logits, weights=0.0, label_smoothing = 0.2)
            level1_aux_net = math_ops.reduce_mean(level1_aux_input, [1, 2], name='level1_aux_pool', keep_dims=True)
            level1_aux_net = layers.conv2d(
                  level1_aux_net,
                  LEVEL1_CLASS, [1, 1],
                  activation_fn=None,
                  normalizer_fn=None,
                  scope='level1_aux_conv')
            level1_aux_logits = tf.squeeze(level1_aux_net)
            loss_level1 = tf.losses.softmax_cross_entropy(onehot_labels = level1_labels, logits = level1_aux_logits, weights=0., label_smoothing = 0.2)

            #variables_to_restore_from_checkpoint = slim.get_variables_to_restore(exclude = variables_to_exclude)
            # Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced, e.x. label smothing
            end_points['logits_output_squeezed'] = tf.squeeze(logits)
            loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = end_points['logits_output_squeezed'], label_smoothing = 0.05)
            total_loss = tf.losses.get_total_loss()    # obtain the regularization losses as well

            # Create the global step for monitoring the learning_rate and training.
            # since supervisor will also create one global_step, so we create n advance in order to feed into exponential_decay
            global_step = tf.train.get_or_create_global_step(graph = graph)

            #Define your exponentially decaying learning rate
            lr = my_exponential_decay(#tf.train.exponential_decay(
                learning_rate = initial_learning_rate,
                global_step = global_step,
                decay_steps = decay_steps,
                decay_rate = learning_rate_decay_factor,
                staircase = True)

            #Now we can define the optimizer that takes on the learning rate
            #optimizer = tf.train.AdamOptimizer(learning_rate = lr)
            #optimizer = tf.train.RMSPropOptimizer(learning_rate = lr)
            optimizer = tf.train.MomentumOptimizer(learning_rate = lr, momentum=momentum)
            
            # Gather update_ops from the first clone. These contain, for example,
            # the updates for the batch_norm variables created by network_fn.
            # update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

            moving_average_variables = slim.get_model_variables()
            variable_averages = tf.train.ExponentialMovingAverage(moving_average_decay, global_step)
            # Use an alternative set of update ops in addition to the default updates:
            tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, variable_averages.apply(moving_average_variables))

            #variables_to_restore_checkpoint = (ops.get_collection(ops.GraphKeys.GLOBAL_VARIABLES) + ops.get_collection(ops.GraphKeys.SAVEABLE_OBJECTS))
            
            #Create the train_op.
            accumulate_factor = tf.constant([1./ACCUMULATE_STEP])
            train_op, accum_ops, zero_ops = my_create_train_op(total_loss, optimizer, False, accumulate_factor)
            
            variables_to_restore_checkpoint = slim.get_variables_to_restore()
            
            #Create the train_op.
            #train_op = slim.learning.create_train_op(total_loss, optimizer, summarize_gradients=False)

            #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
            predictions = tf.argmax(tf.squeeze(end_points['predictions']), 1)
            probabilities = end_points['predictions']
            accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, tf.argmax(one_hot_labels, 1), name='train_accuracy')
            metrics_op = tf.group(accuracy_update)


            #Now finally create all the summaries you need to monitor and group them into one summary op.
            tf.summary.scalar('losses/Total_Loss', total_loss)
            tf.summary.scalar('train/accuracy', accuracy)
            tf.summary.scalar('learning_rate', lr)

            return train_op, accum_ops, zero_ops, global_step, metrics_op, variables_to_restore, variables_to_restore_checkpoint, predictions, lr, accuracy, total_loss

In [11]:
with def_graph.as_default() as graph:
    label_mapping = LabelMapping(CATEGORY_NAME_PATH)
    train_dataset = CdiscountDataset(TRAIN_PATH, 'output_file', label_mapping, TOTAL_EXAMPLES, NUM_CLASS, 8000, BATCH_SIZE, NUM_EPOCHES, True)
   
    batch_images, batch_labels, batch_level0_labels, batch_level1_labels = train_dataset.create_dataset()
   
    with tf.device('/gpu:0'):
        train_op, accum_op, zero_op, global_step, metrics_op, variables_to_restore, variables_to_restore_checkpoint, pred_op, lr, accuracy, total_loss = train_step(batch_images, batch_labels, batch_level0_labels, batch_level1_labels)
         
    summary_op = tf.summary.merge_all()
    # Create a saver that restores only the pre-trained variables.
    # we have change optim, restore all param use pretrained mode
    #pre_train_saver = tf.train.Saver(variables_to_restore)
    
    variables = slim.get_variables_to_restore()
    restore_from_pretrained = tf.contrib.framework.filter_variables(
        variables_to_restore_checkpoint,
        include_patterns=None,
        exclude_patterns=['Momentum'])

    checkpoint_saver = tf.train.Saver(variables_to_restore_checkpoint)
    
    pre_train_saver = tf.train.Saver(restore_from_pretrained)
    #pre_train_saver = tf.train.Saver(variables_to_restore)
    # Define an init function that loads the pretrained checkpoint.
    # sess is the managed session passed by Supervisor
    def load_pretrain(sess):
        pre_train_saver.restore(sess, PRETRAINED_MODEL_PATH)

    # no need for specify local_variables_initializer and tables_initializer, Supervisor will do this via default local_init_op
    # init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(), tf.tables_initializer())
    init_op = tf.group(tf.global_variables_initializer())
    #init_op = tf.group(train_iterator_initializer, val_iterator_initializer, tf.global_variables_initializer())
    
    # Pass the init function to the supervisor.
    # - The init function is called _after_ the variables have been initialized by running the init_op.
    # - use default tf.Saver() for ordinary save and restore
    # - save checkpoint every 1.3 hours(4800)
    # - manage summary in current process by ourselves for memory saving
    # - no need to specify global_step, supervisor will find this automately
    # - initialize order: checkpoint -> local_init_op -> init_op -> init_func
    sv = tf.train.Supervisor(logdir=LOG_PATH, init_fn = load_pretrain, init_op = init_op, summary_op = None, saver = checkpoint_saver, save_model_secs=8000, checkpoint_basename='resnet101_v2_model.ckpt')
    
    final_loss = 0.
    final_accuracy = 0.
    training_state = True

    config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
    #config.gpu_options.allow_growth = True
    with sv.managed_session(config=config) as sess:
    #with sv.prepare_or_wait_for_session(config=tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)) as sess:
        #sess.run(iterator_initalizer)
        # Here sess was either initialized from the pre-trained-checkpoint or
        # recovered from a checkpoint saved in a previous run of this code.
        for step in range(int(num_steps_per_epoch * NUM_EPOCHES)):         
            if sv.should_stop():
                tf_logging.info('Supervisor emit finished!')
                tf_logging.info('Current Loss: %s', loss)
                tf_logging.info('Current Accuracy: %s', accuracy)
                tf_logging.info('Saving current model to disk(maybe invalid).')
                training_state = False
                break

            start_time = time.time()

            if step % 1000 == 0:
                summ = sess.run(summary_op)
                sv.summary_computed(sess, summ)
                if step > EPOCHES_OVER * num_steps_per_epoch:
                    raise StopIteration("over epoches reached.")

            with tf.device('/gpu:0'):
                # accumulate gradient to get bigger batch_size
                sess.run(zero_op)
                for _ in range(1, ACCUMULATE_STEP):
                    sess.run([accum_op, total_loss])
                _, _, _, cur_loss, cur_acc, total_step, cur_lr = sess.run([train_op, accum_op, metrics_op, total_loss, accuracy, global_step, lr])
#                     sess.run([train_op])
            time_elapsed = time.time() - start_time

            if step % 10 == 0:
                final_loss = cur_loss
                final_accuracy = cur_acc
                tf_logging.info('Current Speed: {:5.3f}sec/batch'.format(time_elapsed))
                tf_logging.info('Current Streaming Accuracy: {:5.3f}%'.format(cur_acc*100.))
                tf_logging.info('Current Loss: {:5.3f}'.format(cur_loss))
                tf_logging.info('Epoch %s/%s, Global Step: %s', int(total_step / num_steps_per_epoch + 1), NUM_EPOCHES, total_step)
                tf_logging.info('Current Learning Rate: {}'.format(cur_lr))


        if training_state:
            #We log the final training loss and accuracy
            tf_logging.info('Final Loss: %s', final_loss)
            tf_logging.info('Final Accuracy: %s', final_accuracy)
            # Once all the training has been done, save the log files and checkpoint model
            tf_logging.info('Finished training! Model saved.')
        sv.saver.save(sess, sv.save_path, global_step = sv.global_step)
    

INFO:tensorflow:Restoring parameters from /media/rs/0E06CD1706CD0127/Kapok/kaggle/Resnet/logs101-new/model_finetune/resnet101_v2_model.ckpt-375933
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path /media/rs/0E06CD1706CD0127/Kapok/kaggle/Resnet/logs101-new/resnet101_v2_model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:Current Speed: 18.004sec/batch
INFO:tensorflow:Current Streaming Accuracy: 0.000%
INFO:tensorflow:Current Loss: 6.064
INFO:tensorflow:Epoch 3/12, Global Step: 375935
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.877sec/batch
INFO:tensorflow:Current Streaming Accuracy: 28.281%
INFO:tensorflow:Current Loss: 5.484
INFO:tensorflow:Epoch 3/12, Global Step: 375945
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.818sec/batch
INFO:tensorflow:Current Streaming Accuracy: 28.242%
INFO:tensorflow:Current Los

INFO:tensorflow:Current Speed: 0.875sec/batch
INFO:tensorflow:Current Streaming Accuracy: 27.346%
INFO:tensorflow:Current Loss: 6.031
INFO:tensorflow:Epoch 3/12, Global Step: 376255
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.857sec/batch
INFO:tensorflow:Current Streaming Accuracy: 27.367%
INFO:tensorflow:Current Loss: 5.598
INFO:tensorflow:Epoch 3/12, Global Step: 376265
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.869sec/batch
INFO:tensorflow:Current Streaming Accuracy: 27.387%
INFO:tensorflow:Current Loss: 5.799
INFO:tensorflow:Epoch 3/12, Global Step: 376275
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.854sec/batch
INFO:tensorflow:Current Streaming Accuracy: 27.382%
INFO:tensorflow:Current Loss: 5.578
INFO:tensorflow:Epoch 3/12, Global Step: 376285
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0

INFO:tensorflow:Epoch 3/12, Global Step: 376585
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.865sec/batch
INFO:tensorflow:Current Streaming Accuracy: 27.080%
INFO:tensorflow:Current Loss: 5.768
INFO:tensorflow:Epoch 3/12, Global Step: 376595
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.863sec/batch
INFO:tensorflow:Current Streaming Accuracy: 27.094%
INFO:tensorflow:Current Loss: 5.698
INFO:tensorflow:Epoch 3/12, Global Step: 376605
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:global_step/sec: 1.15834
INFO:tensorflow:Current Speed: 0.863sec/batch
INFO:tensorflow:Current Streaming Accuracy: 27.075%
INFO:tensorflow:Current Loss: 6.024
INFO:tensorflow:Epoch 3/12, Global Step: 376615
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.893sec/batch
INFO:tensorflow:Current Streaming Accuracy: 27.037%
INFO:tensorflow:Current Los

INFO:tensorflow:Current Speed: 0.870sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.938%
INFO:tensorflow:Current Loss: 5.601
INFO:tensorflow:Epoch 3/12, Global Step: 376925
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 1.307sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.935%
INFO:tensorflow:Current Loss: 5.771
INFO:tensorflow:Epoch 3/12, Global Step: 376935
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.881sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.921%
INFO:tensorflow:Current Loss: 5.828
INFO:tensorflow:Epoch 3/12, Global Step: 376945
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.893sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.927%
INFO:tensorflow:Current Loss: 6.271
INFO:tensorflow:Epoch 3/12, Global Step: 376955
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0

INFO:tensorflow:Epoch 3/12, Global Step: 377255
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.845sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.922%
INFO:tensorflow:Current Loss: 5.783
INFO:tensorflow:Epoch 3/12, Global Step: 377265
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.866sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.924%
INFO:tensorflow:Current Loss: 5.995
INFO:tensorflow:Epoch 3/12, Global Step: 377275
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.871sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.920%
INFO:tensorflow:Current Loss: 5.699
INFO:tensorflow:Epoch 3/12, Global Step: 377285
INFO:tensorflow:Current Learning Rate: 9.26582288229838e-05
INFO:tensorflow:Current Speed: 0.881sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.931%
INFO:tensorflow:Current Loss: 6.083
INFO:tensorflow:Epoch 3/12, Glob

...

INFO:tensorflow:Current Speed: 0.851sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.623%
INFO:tensorflow:Current Loss: 5.922
INFO:tensorflow:Epoch 4/12, Global Step: 443995
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.856sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.624%
INFO:tensorflow:Current Loss: 5.918
INFO:tensorflow:Epoch 4/12, Global Step: 444005
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.852sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.623%
INFO:tensorflow:Current Loss: 6.291
INFO:tensorflow:Epoch 4/12, Global Step: 444015
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.858sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.623%
INFO:tensorflow:Current Loss: 5.636
INFO:tensorflow:Epoch 4/12, Global Step: 444025
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Spee

INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.866sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.622%
INFO:tensorflow:Current Loss: 6.237
INFO:tensorflow:Epoch 4/12, Global Step: 444335
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.848sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.622%
INFO:tensorflow:Current Loss: 5.545
INFO:tensorflow:Epoch 4/12, Global Step: 444345
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.861sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.622%
INFO:tensorflow:Current Loss: 5.855
INFO:tensorflow:Epoch 4/12, Global Step: 444355
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.873sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.622%
INFO:tensorflow:Current Loss: 5.904
INFO:tensorflow:Epoch 4/12, Global Step: 444365
INFO:tensorflow:Current Lear

INFO:tensorflow:Epoch 4/12, Global Step: 444665
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.867sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.621%
INFO:tensorflow:Current Loss: 5.892
INFO:tensorflow:Epoch 4/12, Global Step: 444675
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.813sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.621%
INFO:tensorflow:Current Loss: 5.707
INFO:tensorflow:Epoch 4/12, Global Step: 444685
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.840sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.621%
INFO:tensorflow:Current Loss: 6.196
INFO:tensorflow:Epoch 4/12, Global Step: 444695
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.839sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.621%
INFO:tensorflow:Current Loss: 5.907
INFO:tensorflow:Epoch 4/12, 

INFO:tensorflow:Current Loss: 5.605
INFO:tensorflow:Epoch 4/12, Global Step: 445005
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.884sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.620%
INFO:tensorflow:Current Loss: 5.730
INFO:tensorflow:Epoch 4/12, Global Step: 445015
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.899sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.621%
INFO:tensorflow:Current Loss: 5.944
INFO:tensorflow:Epoch 4/12, Global Step: 445025
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.881sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.621%
INFO:tensorflow:Current Loss: 5.787
INFO:tensorflow:Epoch 4/12, Global Step: 445035
INFO:tensorflow:Current Learning Rate: 7.944284152472392e-05
INFO:tensorflow:Current Speed: 0.891sec/batch
INFO:tensorflow:Current Streaming Accuracy: 26.621%
INFO:tensorflow:Current Loss

KeyboardInterrupt: 